
---

# 🧪 Лабораторная работа №3

# Выполнил: Шамсутдинов Рустам БВТ2201

## 🎯 Цель

Сравнить **ASR-движки** — *Whisper*, *Vosk* и *Giga AM* —
по метрикам **WER**, **CER**, **RTF**,
а также оценить влияние **bias prompt (контекста)** в *Whisper*.

---

## 📋 Что нужно сделать

1. **Подготовить данные**

   * 10 аудиофайлов формата **WAV** (16 кГц, mono)
   * Эталонные транскрипты в `.txt`

2. **Провести распознавание речи**

   * Прогнать каждое аудио через:

     * 🟢 Whisper
     * 🔵 Vosk
     * 🟣 Giga AM

3. **Посчитать метрики**

   * **WER (Word Error Rate)**
   * **CER (Character Error Rate)**
   * **RTF (Real-Time Factor)**
   * Для каждого файла вычислить и сравнить результаты.

4. **Оценить влияние bias prompt**

   * Записать **5–10 коротких аудио** с доменными терминами.
   * Для *Whisper* выполнить два прогона:

     1. Без `prompt`
     2. С `initial_prompt`, содержащим перечень терминов
   * Сравнить значения **WER/CER** в обоих случаях.

---

## 🔗 Полезные ссылки и материалы

* [SYSTRAN Faster Whisper](https://github.com/SYSTRAN/faster-whisper)
* [Salute Developers GigaAM](https://github.com/salute-developers/GigaAM)
* [Vosk API](https://github.com/alphacep/vosk-api)
* [Hugging Face Audio Course — Chapter 5: Evaluation](https://huggingface.co/learn/audio-course/ru/chapter5/evaluation)

---


# Task 1

In [ ]:
!pip install faster-whisper
!pip install evaluate
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 43.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 48.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 64.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21

In [6]:
from faster_whisper import WhisperModel
from evaluate import load
import wave
import time
import pandas as pd
import os

model_size = "large-v3"                
device = "cuda"                       
compute_type = "float16"             

model = WhisperModel(model_size, device=device, compute_type=compute_type)

wer_metric = load("wer")
cer_metric = load("cer")

ids = [8, 22, 24, 34, 81, 82, 93, 157, 171, 245]

base_path = '/kaggle/input/mtuci-audiolab3task-1'

results = []

def get_wav_duration(path):
    with wave.open(path, 'rb') as wf:
        frames = wf.getnframes()
        rate = wf.getframerate()
        return frames / float(rate)

for audio_id in ids:
    audio_filename = os.path.join(base_path, f'audio_{audio_id}.wav')
    transcript_filename = os.path.join(base_path, f'transcript_{audio_id}.txt')

    with open(transcript_filename, 'r', encoding='utf-8') as f:
        reference = f.read().strip()

    audio_duration = get_wav_duration(audio_filename)

    t0 = time.time()
    segments, _ = model.transcribe(audio_filename, word_timestamps=True)
    t1 = time.time()
    decode_time = t1 - t0

    words = []
    for segment in segments:
        for w in segment.words:
            words.append(w.word)
    prediction = " ".join(words).strip()

    wer = wer_metric.compute(references=[reference], predictions=[prediction])
    cer = cer_metric.compute(references=[reference], predictions=[prediction])

    rtf = decode_time / audio_duration

    print(f"ID {audio_id}: duration={audio_duration:.2f}s, decode_time={decode_time:.2f}s, RTF={rtf:.3f}")
    print(f"WER={wer:.3f}, CER={cer:.3f}")
    print("Reference:", reference)
    print("Prediction:", prediction)
    print("-" * 60)

    results.append({
        "audio_id": audio_id,
        "audio_filename": audio_filename,
        "duration_s": audio_duration,
        "decode_time_s": decode_time,
        "rtf": rtf,
        "wer": wer,
        "cer": cer,
        "reference": reference,
        "prediction": prediction,
    })

df = pd.DataFrame(results)
csv_out = "whisper_asr_eval_results.csv"
df.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Saved results to {csv_out}")

if not df.empty:
    avg_rtf = df["rtf"].mean()
    avg_wer = df["wer"].mean()
    avg_cer = df["cer"].mean()
    print(f"AVERAGE: RTF={avg_rtf:.3f}, WER={avg_wer:.3f}, CER={avg_cer:.3f}")


ID 8: duration=36.34s, decode_time=0.69s, RTF=0.019
WER=0.060, CER=0.202
Reference: я знаю я по стеночке еду я тоже пошел дели они мне интересно что он делает так я в главную хижину пошел он за мной я туда который завезу водил давай давай давай давай и они ничего не делает бля подломал немножко подломал за тобой бегать нет они тоже 70
Prediction: я  знаю  я  по  стеночке  еду  я  тоже  пошел  дели  они  мне  интересно  что  он  делает  так  я  в  главную  хижину  пошел  он  за  мной  я  туда  который  завезу  водил  давай  давай  давай  давай  лёня  ничего  не  делает  бля  подломал  немножко  подломал  за  тобой  бегать  нет  ни  тоже  70
------------------------------------------------------------
ID 22: duration=38.70s, decode_time=0.67s, RTF=0.017
WER=0.262, CER=0.350
Reference: раз уже висишь я за светкой так я не вижу здесь аккуратно иду вовтихо стиль ниндзя так сеткой на всех парах тебе хуяр он ставил вас тебя да и не видела вроде не ставил можешь меня чуть починить на самом дел

In [ ]:
!pip install gigaam[longform]
!pip install evaluate
!pip install jiwer

  Using cached gigaam-0.1.0-py3-none-any.whl.metadata (10 kB)
Using cached gigaam-0.1.0-py3-none-any.whl (22 kB)


In [ ]:
import os
import time
import wave
import pandas as pd
from evaluate import load
import gigaam
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")
model_name = "v2_rnnt"        
use_flash = False             
base_path = "/kaggle/input/mtuci-audiolab3task-1"
ids = [8, 22, 24, 34, 81, 82, 93, 157, 171, 245]

def get_wav_duration(path):
    with wave.open(path, 'rb') as wf:
        frames = wf.getnframes()
        rate = wf.getframerate()
        return frames / float(rate)


print("Loading GigaAM model:", model_name)
model = gigaam.load_model(model_name, use_flash=use_flash)
print("Model loaded.")

wer_metric = load("wer")
cer_metric = load("cer")

results = []

for audio_id in ids:
    audio_filename = os.path.join(base_path, f"audio_{audio_id}.wav")
    transcript_filename = os.path.join(base_path, f"transcript_{audio_id}.txt")

    with open(transcript_filename, 'r', encoding='utf-8') as f:
        reference = f.read().strip()

    duration = get_wav_duration(audio_filename)

    t0 = time.time()
    recognition_result = model.transcribe_longform(audio_filename)
    t1 = time.time()
    decode_time = t1 - t0

    utterances = []
    for utt in recognition_result:
        text = utt.get("transcription") or utt.get("text") or ""
        text = text.strip()
        if text:
            utterances.append(text)
    prediction = " ".join(utterances).strip()

    wer = wer_metric.compute(references=[reference], predictions=[prediction])
    cer = cer_metric.compute(references=[reference], predictions=[prediction])

    rtf = decode_time / duration if duration > 0 else float("inf")

    print(f"ID {audio_id}: duration={duration:.2f}s, decode_time={decode_time:.2f}s, RTF={rtf:.3f}")
    print(f"WER={wer:.3f}, CER={cer:.3f}")
    print("Reference:", reference)
    print("Prediction:", prediction)
    print("-" * 60)

    results.append({
        "audio_id": audio_id,
        "audio_filename": audio_filename,
        "duration_s": duration,
        "decode_time_s": decode_time,
        "rtf": rtf,
        "wer": wer,
        "cer": cer,
        "reference": reference,
        "prediction": prediction,
    })


df = pd.DataFrame(results)
csv_out = "gigaam_asr_eval_results.csv"
df.to_csv(csv_out, index=False, encoding="utf-8")
print(f"Saved results to {csv_out}")

if not df.empty:
    avg_rtf = df["rtf"].mean()
    avg_wer = df["wer"].mean()
    avg_cer = df["cer"].mean()
    print(f"AVERAGE: RTF={avg_rtf:.3f}, WER={avg_wer:.3f}, CER={avg_cer:.3f}")


Loading GigaAM model: v2_rnnt


/home/sypoo/Mtuci/mtuci-audio/.venv/lib/python3.12/site-packages/gigaam/__init__.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, ma

Model loaded.
ID 8: duration=36.34s, decode_time=3.96s, RTF=0.109
WER=0.440, CER=0.357
Reference: я знаю я по стеночке еду я тоже пошел дели они мне интересно что он делает так я в главную хижину пошел он за мной я туда который завезу водил давай давай давай давай и они ничего не делает бля подломал немножко подломал за тобой бегать нет они тоже 70
Prediction: я тоже пошел где леня мне интересно что он делает так я в главную хижину пошел он за мной давай давай давай давай леня ничего не делает бля подломал немножко подломал сильно он за тобой бегает нет а у лени тоже семьдесят
------------------------------------------------------------
ID 22: duration=38.70s, decode_time=3.10s, RTF=0.080
WER=0.410, CER=0.248
Reference: раз уже висишь я за светкой так я не вижу здесь аккуратно иду вовтихо стиль ниндзя так сеткой на всех парах тебе хуяр он ставил вас тебя да и не видела вроде не ставил можешь меня чуть починить на самом деле он же не гоняется за давай вот он уронил около меня хорошо как

In [1]:
!pip install vosk
!pip install evaluate
!pip install jiwer


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.8 MB/s eta 0:00:0000:0100:01
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=9e326f9052ebe068bf9042adc0203eeaa82e71173e3d969eae9431e0033bba40
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 29.6 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.


In [8]:
import os
import wave
import time
import json
import pandas as pd
from evaluate import load
from vosk import Model, KaldiRecognizer, SetLogLevel


MODEL = "vosk-model-ru-0.42"
base_path = "/kaggle/input/mtuci-audiolab3task-1"
ids = [8, 22, 24, 34, 81, 82, 93, 157, 171, 245]
csv_out = "vosk_asr_eval_results.csv"

model = Model(model_name=MODEL)

wer_metric = load("wer")
cer_metric = load("cer")

results = []

def get_wav_duration(path):
    with wave.open(path, "rb") as wf:
        frames = wf.getnframes()
        rate = wf.getframerate()
        return frames / float(rate)

def transcribe_with_vosk(audio_path, model, read_chunk=4000):
    wf = wave.open(audio_path, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)  
    rec.SetPartialWords(False)

    words = []
    t0 = time.time()

    while True:
        data = wf.readframes(read_chunk)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            text = res.get("text", "").strip()
            if text:
                words.extend(text.split())
    final_res = json.loads(rec.FinalResult())
    final_text = final_res.get("text", "").strip()
    if final_text:
        words.extend(final_text.split())

    t1 = time.time()
    decode_time = t1 - t0
    prediction = " ".join(words).strip()

    wf.close()
    return prediction, decode_time

for audio_id in ids:
    audio_filename = os.path.join(base_path, f"audio_{audio_id}.wav")
    transcript_filename = os.path.join(base_path, f"transcript_{audio_id}.txt")

    with open(transcript_filename, "r", encoding="utf-8") as f:
        reference = f.read().strip()

    audio_duration = get_wav_duration(audio_filename)

    prediction, decode_time = transcribe_with_vosk(audio_filename, model)

    wer = wer_metric.compute(references=[reference], predictions=[prediction])
    cer = cer_metric.compute(references=[reference], predictions=[prediction])

    rtf = decode_time / audio_duration if audio_duration > 0 else float("inf")

    print(f"ID {audio_id}: duration={audio_duration:.2f}s, decode_time={decode_time:.2f}s, RTF={rtf:.3f}")
    print(f"WER={wer:.3f}, CER={cer:.3f}")
    print("Reference:", reference)
    print("Prediction:", prediction)
    print("-" * 60)

    results.append({
        "audio_id": audio_id,
        "audio_filename": audio_filename,
        "duration_s": audio_duration,
        "decode_time_s": decode_time,
        "rtf": rtf,
        "wer": wer,
        "cer": cer,
        "reference": reference,
        "prediction": prediction,
    })
df = pd.DataFrame(results)
df.to_csv(csv_out, index=False, encoding="utf-8")
print(f"Saved results to {csv_out}")

if not df.empty:
    avg_rtf = df["rtf"].mean()
    avg_wer = df["wer"].mean()
    avg_cer = df["cer"].mean()
    print(f"AVERAGE: RTF={avg_rtf:.3f}, WER={avg_wer:.3f}, CER={avg_cer:.3f}")


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /root/.cache/vosk/vosk-model-ru-0.42/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /root/.cache/vosk/vosk-model-ru-0.42/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:297) Loading words from /root/.cache/vosk/vosk-model-ru-0.42/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winf

ID 8: duration=36.34s, decode_time=5.41s, RTF=0.149
WER=0.360, CER=0.206
Reference: я знаю я по стеночке еду я тоже пошел дели они мне интересно что он делает так я в главную хижину пошел он за мной я туда который завезу водил давай давай давай давай и они ничего не делает бля подломал немножко подломал за тобой бегать нет они тоже 70
Prediction: я знаю что я по стеночке иду я тоже пошёл дядя лёня мне интересно что он делает так я в главную хижину пошёл он за мной я это даже который завёз заводил давай давай давай давай лёня ничего не делает бля отломал немножко под ломал сильно за тобой бегать нет аллергии тоже семьдесят
------------------------------------------------------------
ID 22: duration=38.70s, decode_time=7.11s, RTF=0.184
WER=0.393, CER=0.244
Reference: раз уже висишь я за светкой так я не вижу здесь аккуратно иду вовтихо стиль ниндзя так сеткой на всех парах тебе хуяр он ставил вас тебя да и не видела вроде не ставил можешь меня чуть починить на самом деле он же не гоняетс

# Task 2

In [9]:
!pip install faster-whisper

!pip install evaluate
!pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 40.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 43.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 42.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.38.1 requires pydantic<2.12

In [15]:
from faster_whisper import WhisperModel
from evaluate import load
import wave
import time
import pandas as pd
import os

model_size = "large-v3"                
device = "cuda"                       
compute_type = "float16"             

model = WhisperModel(model_size, device=device, compute_type=compute_type)

wer_metric = load("wer")
cer_metric = load("cer")

ids = range(1, 5 + 1)

base_path = '/kaggle/input/mtuci-audio-lab3-task-2'

results = []

def get_wav_duration(path):
    with wave.open(path, 'rb') as wf:
        frames = wf.getnframes()
        rate = wf.getframerate()
        return frames / float(rate)

for audio_id in ids:
    audio_filename = os.path.join(base_path, f'output{audio_id}.wav')
    transcript_filename = os.path.join(base_path, f'text{audio_id}.txt')

    with open(transcript_filename, 'r', encoding='utf-8') as f:
        reference = f.read().strip()

    audio_duration = get_wav_duration(audio_filename)

    t0 = time.time()
    segments, _ = model.transcribe(audio_filename, word_timestamps=True)
    t1 = time.time()
    decode_time = t1 - t0

    words = []
    for segment in segments:
        for w in segment.words:
            words.append(w.word)
    prediction = " ".join(words).strip()

    wer = wer_metric.compute(references=[reference], predictions=[prediction])
    cer = cer_metric.compute(references=[reference], predictions=[prediction])

    rtf = decode_time / audio_duration

    print(f"ID {audio_id}: duration={audio_duration:.2f}s, decode_time={decode_time:.2f}s, RTF={rtf:.3f}")
    print(f"WER={wer:.3f}, CER={cer:.3f}")
    print("Reference:", reference)
    print("Prediction:", prediction)
    print("-" * 60)

    results.append({
        "audio_id": audio_id,
        "audio_filename": audio_filename,
        "duration_s": audio_duration,
        "decode_time_s": decode_time,
        "rtf": rtf,
        "wer": wer,
        "cer": cer,
        "reference": reference,
        "prediction": prediction,
    })

df = pd.DataFrame(results)
csv_out = "whisper_asr_domain_eval_results.csv"
df.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Saved results to {csv_out}")

if not df.empty:
    avg_rtf = df["rtf"].mean()
    avg_wer = df["wer"].mean()
    avg_cer = df["cer"].mean()
    print(f"AVERAGE: RTF={avg_rtf:.3f}, WER={avg_wer:.3f}, CER={avg_cer:.3f}")


ID 1: duration=3.33s, decode_time=0.76s, RTF=0.227
WER=0.000, CER=0.086
Reference: Нужно протестировать на продакшене.
Prediction: Нужно  протестировать  на  продакшене.
------------------------------------------------------------
ID 2: duration=4.66s, decode_time=0.68s, RTF=0.146
WER=0.600, CER=0.140
Reference: Попробуй локально развернуть через докерконтейнер.
Prediction: Попробую  локально  развернуть  через  докер -контейнер.
------------------------------------------------------------
ID 3: duration=3.48s, decode_time=0.67s, RTF=0.193
WER=0.000, CER=0.139
Reference: Нужно ускорить запрос к базе данных.
Prediction: Нужно  ускорить  запрос  к  базе  данных.
------------------------------------------------------------
ID 4: duration=3.53s, decode_time=0.69s, RTF=0.195
WER=0.833, CER=0.439
Reference: Продакшен упал из-за ошибки в датацентре.
Prediction: Production  упал  из -за  ошибки  в  дата -центре.
------------------------------------------------------------
ID 5: duration=3.29s,

In [17]:
from faster_whisper import WhisperModel
from evaluate import load
import wave
import time
import pandas as pd
import os

model_size = "large-v3"                
device = "cuda"                       
compute_type = "float16"             

model = WhisperModel(model_size, device=device, compute_type=compute_type)

wer_metric = load("wer")
cer_metric = load("cer")

ids = range(1, 5 + 1)

base_path = '/kaggle/input/mtuci-audio-lab3-task-2'

results = []

initial_prompt = """Это аудио с фразами, которые часто используют программисты.
Можно встретить такие термины, как продакшен, тестирование, локально, докерконтейнер, запрос, база данных, датацентр, скрипт, питон."""

def get_wav_duration(path):
    with wave.open(path, 'rb') as wf:
        frames = wf.getnframes()
        rate = wf.getframerate()
        return frames / float(rate)

for audio_id in ids:
    audio_filename = os.path.join(base_path, f'output{audio_id}.wav')
    transcript_filename = os.path.join(base_path, f'text{audio_id}.txt')

    with open(transcript_filename, 'r', encoding='utf-8') as f:
        reference = f.read().strip()

    audio_duration = get_wav_duration(audio_filename)

    t0 = time.time()
    segments, _ = model.transcribe(
        audio_filename,
        word_timestamps=True,
        initial_prompt=initial_prompt
    )
    t1 = time.time()
    decode_time = t1 - t0

    words = []
    for segment in segments:
        for w in segment.words:
            words.append(w.word)
    prediction = " ".join(words).strip()

    wer = wer_metric.compute(references=[reference], predictions=[prediction])
    cer = cer_metric.compute(references=[reference], predictions=[prediction])

    rtf = decode_time / audio_duration

    print(f"ID {audio_id}: duration={audio_duration:.2f}s, decode_time={decode_time:.2f}s, RTF={rtf:.3f}")
    print(f"WER={wer:.3f}, CER={cer:.3f}")
    print("Reference:", reference)
    print("Prediction:", prediction)
    print("-" * 60)

    results.append({
        "audio_id": audio_id,
        "audio_filename": audio_filename,
        "duration_s": audio_duration,
        "decode_time_s": decode_time,
        "rtf": rtf,
        "wer": wer,
        "cer": cer,
        "reference": reference,
        "prediction": prediction,
    })

df = pd.DataFrame(results)
csv_out = "whisper_asr_domain_with_initial_prompt_eval_results.csv"
df.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Saved results to {csv_out}")

if not df.empty:
    avg_rtf = df["rtf"].mean()
    avg_wer = df["wer"].mean()
    avg_cer = df["cer"].mean()
    print(f"AVERAGE: RTF={avg_rtf:.3f}, WER={avg_wer:.3f}, CER={avg_cer:.3f}")


ID 1: duration=3.33s, decode_time=0.75s, RTF=0.224
WER=0.000, CER=0.086
Reference: Нужно протестировать на продакшене.
Prediction: Нужно  протестировать  на  продакшене.
------------------------------------------------------------
ID 2: duration=4.66s, decode_time=0.67s, RTF=0.145
WER=0.000, CER=0.080
Reference: Попробуй локально развернуть через докерконтейнер.
Prediction: Попробуй  локально  развернуть  через  докерконтейнер.
------------------------------------------------------------
ID 3: duration=3.48s, decode_time=0.67s, RTF=0.194
WER=0.000, CER=0.139
Reference: Нужно ускорить запрос к базе данных.
Prediction: Нужно  ускорить  запрос  к  базе  данных.
------------------------------------------------------------
ID 4: duration=3.53s, decode_time=0.68s, RTF=0.193
WER=0.333, CER=0.146
Reference: Продакшен упал из-за ошибки в датацентре.
Prediction: Продакшен  упал  из -за  ошибки  в  датацентре.
------------------------------------------------------------
ID 5: duration=3.29s, deco